In [13]:
import os
from dotenv import load_dotenv
load_dotenv()
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_URI = os.getenv("NEO4J_URI")

In [14]:
from neo4j import GraphDatabase
uri = NEO4J_URI
username = NEO4J_USERNAME
password = NEO4J_PASSWORD
driver = GraphDatabase.driver(uri, auth=(username, password))
try:
   driver.verify_connectivity()
   print("Connection successful!")
except Exception as e:
   print(f"Failed to connect to Neo4j: {e}")

Connection successful!


In [15]:
# Return the team that won the most number of World Cup matches
query = """
MATCH (t:Team)-[:WON]->(m:Match)-[:PART_OF]->(:Tournament {name: "FIFA World Cup"})
RETURN t.name AS Team, COUNT(m) AS MatchesWon
ORDER BY MatchesWon DESC
LIMIT 1
"""
records, summary, keys = driver.execute_query(query, database_="neo4j")

In [16]:
for record in records:
   print(record.data())

{'Team': 'Brazil', 'MatchesWon': 76}


In [17]:
query = """
MATCH (t:Team)-[:WON]->(m:Match)-[:PART_OF]->(:Tournament {name: $tournament})
RETURN t.name AS Team, COUNT(m) AS MatchesWon
ORDER BY MatchesWon DESC
LIMIT $limit
"""

In [18]:
records, summary, keys = driver.execute_query(
   query, database_="neo4j", tournament="UEFA Euro", limit=3,
)
for record in records:
   print(record.data())

{'Team': 'Germany', 'MatchesWon': 30}
{'Team': 'Spain', 'MatchesWon': 28}
{'Team': 'France', 'MatchesWon': 23}


In [19]:
def add_new_coach(driver, coach_name, team_name, start_date, end_date):
   query = """
   MATCH (t:Team {name: $team_name})
   CREATE (c:Coach {name: $coach_name})
   CREATE (c)-[r:COACHES]->(t)
   SET r.start_date = $start_date
   SET r.end_date = $end_date
   """
   result = driver.execute_query(
       query,
       database_="neo4j",
       coach_name=coach_name,
       team_name=team_name,
       start_date=start_date,
       end_date=end_date
   )
   summary = result.summary
   print(f"Added new coach: {coach_name} for existing team {team_name} starting from {start_date}")
   print(f"Nodes created: {summary.counters.nodes_created}")
   print(f"Relationships created: {summary.counters.relationships_created}")

In [20]:
from neo4j.time import DateTime
add_new_coach(
   driver=driver,
   coach_name="Lionel Scaloni",
   team_name="Argentina",
   start_date=DateTime(2018, 6, 1),
   end_date=None
)

Added new coach: Lionel Scaloni for existing team Argentina starting from 2018-06-01T00:00:00.000000000
Nodes created: 1
Relationships created: 1


In [21]:
def top_goal_scorers(tx, tournament, limit):
   query = """
   MATCH (p:Player)-[s:SCORED_IN]->(m:Match)-[PART_OF]->(t:Tournament)
   WHERE t.name = $tournament
   RETURN p.name AS Player, COUNT(s) AS Goals
   ORDER BY Goals DESC
   LIMIT $limit
   """
   result = tx.run(query, tournament=tournament, limit=limit)
   return [record.data() for record in result]

In [22]:
with driver.session() as session:
   result = session.execute_read(top_goal_scorers, "FIFA World Cup", 5)
   for record in result:
       print(record)

{'Player': 'Miroslav Klose', 'Goals': 16}
{'Player': 'Ronaldo', 'Goals': 15}
{'Player': 'Gerd Müller', 'Goals': 14}
{'Player': 'Lionel Messi', 'Goals': 13}
{'Player': 'Just Fontaine', 'Goals': 13}
